# 04 - Molecular similarity in RDKit

Authored by: *Fredrik Svensson, Oliver Scott*

Edited by: *Florion Peni*

Please ensure you complete [`02_rdkit_introduction.ipynb`](02_rdkit_introduction.ipynb) and [`03_rdkit_substructure.ipynb`](03_rdkit_substructure.ipynb) before proceeding with the content in this notebook. If you are not familiar with the Python basics, make sure to go through [`01_python_introduction.ipynb`](01_python_introduction.ipynb) first.

*Note: This notebook is designed to be run sequentially. Lower code cells will not execute correctly if the cells above them have not also been executed.*

This notebook introduces the concept of molecular similarity and explains how it can be quantified.

## Contents

- [Molecular similarity](#Molecular-similarity)
- [Molecular representation](#Molecular-representation)
- [Molecular fingerprints](#Molecular-fingerprints)
- [Molecular similarity measures](#Molecular-similarity-measures)
- [Generating fingerprints](#Generating-fingerprints)
- [Calculating similarity](#Calculating-similarity)
- [Selecting diverse molecules with the MaxMin algorithm](#Selecting-diverse-molecules-with-the-MaxMin-algorithm)
- [Discussion](#Discussion)

----

## Molecular similarity

Molecular similarity is one of the most important concepts in cheminformatics, referring to the comparison of molecules based on either their **structural** or **functional** properties. This comparison has many applications, particularly in identifying new molecules with desired properties and biological activities.

The ***molecular similarity principle*** states that structurally similar molecules often share similar properties and biological activities, a concept encapsulated in ***structure-activity relationship (SAR) studies***. In virtual screening, this principle is leveraged to identify potential active molecules.

> Given a set of molecules with known biological activity, similarity searching can be used to find additional candidates.

Although the idea of molecular similarity is conceptually simple, it is a complex and multifaceted concept in practice. The way similarity is measured depends on the application and requires three key components:

1. A **molecular representation** that encodes molecular structure or chemical features.
2. A **weighting** of the representation's features, *if applicable*.
3. A **similarity function or coefficient** to combine information and calculate a similarity score.

---

## Molecular representation

The choice of molecular representation has a significant impact on similarity calculations. Different representations are suited to different applications.

### Molecular descriptors

**1D descriptors**

Examples: molecular weight, logP, number of rotatable bonds, etc.

- These provide a single value for the entire molecule, thus they are also known as global descriptors.
- Often combined with 2D fingerprints to enhance representation.
- Do not reflect local regions of similarity.

**2D descriptors**

Examples: Molecular graphs, fragments, local atom environments, etc.

- These represent localised molecular features.
- They are high-dimensional descriptors as they encode many features per molecule.
- Often referred to as *fingerprints* and commonly used for similarity searching and virtual screening.

**3D descriptors**

Examples: Shape, stereochemistry, etc.

- Can encode spatial information but depend on the conformation used.
- More complex and less robust than 2D descriptors due to the many possible conformations a molecule can adopt.

**Biological similarity**
- Requires experimental activity data.
- Often independent of molecular structure.
- Biological fingerprints may encode activity against various biological targets.

In [`02_rdkit_introduction.ipynb`](02_rdkit_introduction.ipynb), we explored how to calculate some molecular descriptors. For a full list of available descriptors in RDKit, refer to the [documentation](https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors). In this notebook, we will focus on 2D and 3D descriptors, particularly molecular fingerprints.

----

## Molecular fingerprints

Molecular fingerprints encode the structure of a molecule into a vector of binary digits (*bits*). Each bit indicates the presence (1) or absence (0) of a specific feature, such as a particular substructure. Comparing these fingerprints using molecular similarity metrics allows us to calculate the similarity between two molecules.

### Structural keys

The earliest type of fingerprint, a **structural key**, is a bit vector where each bit corresponds to a predefined structural feature (encoded by **SMARTS patterns**). For example, the Molecular ACCess System (MACCS) key encodes 166 predefined substructures. *MACCS keys are available in RDKit.*

Structural keys are interpretable but less generalisable than modern approaches.

### Circular fingerprints (Morgan, ECFP)

**Circular fingerprints**, such as those generated by the *Morgan* algorithm, encode circular environments around each atom. The fingerprint’s radius determines how many neighboring atoms and bonds are included in the circular environment.

<i></i>
<a href="https://chembioinfo.wordpress.com/2011/10/30/revisiting-molecular-hashed-fingerprints/">
    <img src="https://chembioinfo.files.wordpress.com/2011/10/signature2.jpg" alt="Circular Fingerprints" style="width: 800px;">
</a>
<i></i>

[Image source](https://chembioinfo.files.wordpress.com/2011/10/signature2.jpg)

These environments are hashed into a numeric representation that determines which bits in the fingerprint are set. To ensure fixed-length fingerprints, they are often *folded* to a desired length.

*Extended Connectivity Fingerprints (ECFPs)*, based on a variant of the Morgan algorithm, are widely used in cheminformatics.

Knowing exactly how they are generated is beyond the scope of this tutorial. For more details, see the [RDKit documentation on Morgan fingerprints](http://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints) or [this publication](https://pubs.acs.org/doi/abs/10.1021/ci100050t).

---

## Molecular similarity measures

The choice of similarity metric is as important as the molecular representation. Different metrics emphasise different aspects of similarity. For binary fingerprints, two of the most common metrics are the **Tanimoto** and **Dice coefficients**. 
These metrics form the basis for comparing molecular fingerprints and assessing the similarity between molecules. The choice of metric depends on the specific application.

#### Tanimoto coefficient 

$$T_{c}(A,B)=\frac{c}{a+b-c}$$

#### Dice coefficient 

$$D_{c}(A,B)=\frac{c}{\frac{1}{2}(a + b)}$$

- **a**: Number of features present in molecule A (*on-bits*).
- **b**: Number of features present in molecule B (*on-bits*).
- **c**: Number of features shared between molecules A and B (*intersection*).

----

## Generating fingerprints

RDKit can be used to compute molecular fingerprints.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

smiles = 'c1cccnc1C'  # SMILES for 2-methylpyridine

# Convert the SMILES string to an RDKit `Mol` object and display it
mol = Chem.MolFromSmiles(smiles)
mol

Now, let's generate a fingerprint for this particular molecule. We will calculate a **Morgan fingerprint**, which is conceptually similar to **ECFPs**. 

**Morgan fingerprints** can be computed in two forms:

1. as an **integer vector** (unfolded), where each feature is represented by its unique hash,
2. or as a **bit vector** (folded), where the fingerprint is compressed into a fixed-length binary array.

Pay attention to the `bitInfo` argument. RDKit populates dictionaries with details about which structural features in the molecule correspond to the bits set in the fingerprint. This information can be useful for interpreting the fingerprint later on.

In [ ]:
# Generate a Morgan fingerprint with a radius of 2 as an integer vector
bit_info_int = {}
fp_int = AllChem.GetMorganFingerprint(mol, radius=2, bitInfo=bit_info_int)

# Generate a Morgan fingerprint with a radius of 2 as a bit vector of fixed length (2048 bits)
bit_info_bit = {}
fp_bit = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048, bitInfo=bit_info_bit)

# No output is expected from this cell
# You may ignore any `DEPRECATION WARNING` messages

Let's look at the information we can get from these fingerprints.

In [ ]:
# Display properties of the integer vector fingerprint
print('Integer vector fingerprint')
print('Length:', fp_int.GetLength())                           # The total length of the fingerprint (number of unique features)
print('Number of on-bits:', len(fp_int.GetNonzeroElements()))  # The number of features that are actually present (non-zero elements)

# Display properties of the bit vector fingerprint
print('\nBit vector fingerprint')
print('Length:', fp_bit.GetNumBits())                          # The total length of the fingerprint (fixed bit size, e.g., 2048)
print('Number of on-bits:', fp_bit.GetNumOnBits())             # The number of features that are actually present (bits set to 1)

Notice that the length of the integer vector fingerprint is very large! In the *bit vector fingerprint*, we "fold" the fingerprint to reduce its length to a fixed size, making it shorter and more manageable for comparison tasks.

We can also inspect which bits have been set in the fingerprint. For the *integer vector fingerprint*, this is represented as a dictionary where:

- the **keys** are the indices of the bits (corresponding to specific features),
- and the **values** indicate how many times each bit was set.

In [ ]:
fp_int.GetNonzeroElements()

Alternatively, we can also just investigate the `bit_info_int` dictionary we generated earlier.

In [ ]:
bit_info_int # Stores the details in the format (atom index, radius)

Notice that the bit at index `98513984` was set twice. 

RDKit provides a way to visualise a bit using the `Draw` module in combination with the `bit_info_int` dictionary. The latter links each bit to the atom environments that caused it to be set.

In [ ]:
# Retrieve information about the bit from the fingerprint
print("Bit 98513984 info (atom index, radius):", bit_info_int[98513984])

# Visualise the molecular environment corresponding to the specified bit
# `Draw.DrawMorganBit()` highlights the atom environment that caused the bit to be set
img = Draw.DrawMorganBit(mol, bitId=98513984, bitInfo=bit_info_int, useSVG=True)
img

From the output, you can observe that the bit at index `98513984` is set twice in 2-methylpyridine. This occurs due to contributions from **atom 1** and **atom 2**, both at a radius of 1.

##### Exercise 1: Generating fingerprints

Using the code above, analyse the bit at index `4048591891` by completing the following tasks:

1. Find out how many times the bit is set in 2-methylpyridine.
2. Deduce which atom(s) (by index) are responsible for setting this bit.
3. Define the radius at which this bit was set.
4. Visualise the molecular feature corresponding to this bit by drawing an image.

<details>
    <summary>
        Example solution
    </summary>

```python
print("Bit 4048591891 info (atom index, radius):", bit_info_int[4048591891])

img = Draw.DrawMorganBit(mol, bitId=4048591891, bitInfo=bit_info_int, useSVG=True)
img
```

</details>

In [ ]:
# Write your solution code here, adding extra code cells if necessary

## Calculating similarity

Now that we have learned how to compute fingerprints in RDKit, the next step is to use these fingerprints to compare molecules. Calculating molecular similarity forms the foundation for **similarity searching**, a key concept in cheminformatics.

To get started, let's define some molecules that we can use for comparison. We can work with the same set of aromatic compounds we used in the previous notebook to explain basic SMARTS queries.

In [ ]:
# Define aromatic compounds using SMILES
naphthalene = Chem.MolFromSmiles('c12ccccc1cccc2')
benzoxazole = Chem.MolFromSmiles('n1c2ccccc2oc1')
indane = Chem.MolFromSmiles('c1ccc2c(c1)CCC2')
skatole = Chem.MolFromSmiles('CC1=CNC2=CC=CC=C12')
benzene = Chem.MolFromSmiles('c1ccccc1')
quinoline = Chem.MolFromSmiles('n1cccc2ccccc12')

# Define a list holding the RDKit `Mol` objects
my_molecules = [
    naphthalene, 
    benzoxazole,
    indane,
    skatole,
    benzene,
    quinoline
]

# Define a separate list holding the names of the aromatic compounds
labels = [
    'Naphthalene', 
    'Benzoxazole',
    'Indane',
    'Skatole',
    'Benzene',
    'Quinoline'
]

# Visualise the aromatic compounds
img = Draw.MolsToGridImage(my_molecules, legends=labels)
img

Let's calculate fingerprints for a couple of these molecules. 

In [ ]:
query1 = my_molecules[0]  # Napthalene
query2 = my_molecules[1]  # Benzoxazole

# Generate bit vector Morgan fingerprints with a radius of 2
fp1 = AllChem.GetMorganFingerprintAsBitVect(query1, radius=2)
fp2 = AllChem.GetMorganFingerprintAsBitVect(query2, radius=2)

# No output is expected from this cell
# You may ignore any `DEPRECATION WARNING` messages

Now that we have generated the fingerprints for our molecules, the next step is to compute their similarity. For those of you paying close attention, you may have noticed that the **Tanimoto coefficient** is essentially the ratio of the **intersection of the on-bits** to the **union of the on-bits** in both fingerprints.

We can easily compute this in Python by creating a custom Tanimoto similarity function that works with bit vectors.

In [ ]:
def tanimoto(fp1, fp2):
    """
    Calculates the Tanimoto coefficient between two bit vector fingerprints.
    
    Args:
        fp1: RDKit bit vector fingerprint for the first molecule.
        fp2: RDKit bit vector fingerprint for the second molecule.
    
    Returns:
        The Tanimoto similarity coefficient between the two fingerprints.
    """
    # Get the set of on-bits (indexes where the bit is set to 1) for the first fingerprint
    on_bits1 = set(fp1.GetOnBits())
    # Get the set of on-bits for the other fingerprint
    on_bits2 = set(fp2.GetOnBits())
    # Compute common bits set to 1 in both fingerprints
    intersection = len(on_bits1.intersection(on_bits2))
    # Compute total unique bits set to 1 across both fingerprints
    union = len(on_bits1.union(on_bits2))
    # Calculate the Tanimoto coefficient
    return intersection / union

# Calculate the Tanimoto similarity between the two fingerprints and print the result
print('Tanimoto similarity:', tanimoto(fp1, fp2))

Fortunately, RDKit provides efficient built-in implementations of similarity metrics, which are part of the `DataStructs` module. Instead of manually calculating the Tanimoto or Dice coefficients, we can use these optimised functions to save time and ensure accuracy.

In [ ]:
from rdkit.DataStructs import TanimotoSimilarity, DiceSimilarity

print('Tanimoto similarity:', TanimotoSimilarity(fp1, fp2))
print('Dice similarity:', DiceSimilarity(fp1, fp2))

Similarity search is a widely used method in **ligand-based virtual screening**, primarily due to its speed and efficiency. The computational simplicity of similarity calculations makes it an excellent first step for filtering large chemical databases. By narrowing down these databases to a smaller subset of molecules, more computationally intensive methods can then be applied to the refined dataset.

##### Exercise 2: Calculating similarity

Using the aromatic compounds defined earlier, complete the following tasks:

1. Create a function to perform pairwise comparisons across the collection, returning the molecule with the highest similarity score to the query. You can achieve this using a loop to iterate through the collection.
2. Identify the molecule in the collection that is most similar to naphthalene. Does this result match your prediction?

<details>
    <summary>
        Example solution
    </summary>

```python
# Generate Morgan fingerprints as bit vectors for all molecules
fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=2) for mol in my_molecules]

# Define function to find the most similar molecule in the collection to a given query
def find_most_similar(query_index, fingerprints, labels):
    """
    Finds the molecule most similar to the query molecule.
    Args:
        query_index: Index of the query molecule.
        fingerprints: List of fingerprints for all molecules.
        labels: List of molecule names corresponding to the fingerprints.
    Returns:
        A tuple containing the name and similarity score of the most similar molecule.
    """
    query_fp = fingerprints[query_index]
    max_similarity = 0
    most_similar_index = -1
    for i, fp in enumerate(fingerprints):
        if i == query_index:  
            continue  # Skip self-comparison
        similarity = TanimotoSimilarity(query_fp, fp)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_index = i
    return labels[most_similar_index], max_similarity

# Calculate pairwise similarities for selected pairs
similarity_1 = TanimotoSimilarity(fingerprints[0], fingerprints[1])
similarity_2 = TanimotoSimilarity(fingerprints[0], fingerprints[4])
similarity_3 = TanimotoSimilarity(fingerprints[2], fingerprints[5])

# Print pairwise similarities
print("Similarity (Naphthalene vs benzoxazole):", similarity_1)
print("Similarity (Naphthalene vs benzene):", similarity_2)
print("Similarity (Indane vs quinoline):", similarity_3)

# Find the most similar molecule to Naphthalene
most_similar_name, max_similarity = find_most_similar(0, fingerprints, labels)  # Call the function
print("\nMost similar molecule to naphthalene:", most_similar_name)
print("Similarity score:", max_similarity)
```

</details>

In [ ]:
# Add your solution code here

----

## Selecting diverse molecules with the MaxMin algorithm

In some cases, it may be important to select a set of molecules that are as diverse as possible to assess the chemical space of a compound library. One of the most efficient ways to achieve this is by using the **MaxMin algorithm**, described in [this paper](http://eprints.whiterose.ac.uk/3570/).

*The algorithm works in 4 steps, as described below.*

1. **Generate molecular descriptors**: Compute descriptors (e.g. Morgan fingerprints) for all molecules in the dataset, including the query molecules (if provided) and the candidate pool.

2. **Initial seed selection**: If no initial query molecules are provided, select one molecule at random from the candidate pool as the starting point for the *picked set*.

3. **MaxMin selection**: For each subsequent selection, identify the molecule from the candidate pool that has the *maximum value of its minimum distance* to the molecules already in the picked set. This ensures that the selected molecule is as diverse as possible from those already picked.

4. **Repeat until completion**: Continue selecting molecules based on the MaxMin rule until the desired number of molecules is chosen.

RDKit provides an implementation of this algorithm in the `MaxMinPicker` class. Let’s apply it to the set of approved drugs we have been working with in previous notebooks to demonstrate how to select a diverse subset of molecules.

This method is particularly useful for exploring chemical diversity in large datasets, selecting training sets for machine learning models, or identifying representative compounds for downstream analyses.

#### STEP 1: Preparing the molecules and fingerprints

In [ ]:
from rdkit.SimDivFilters.rdSimDivPickers import MaxMinPicker
from rdkit import RDLogger

RDLogger.logger().setLevel(RDLogger.CRITICAL)  # Suppress `DEPRECATION WARNING` messages

approved_drugs_file = 'data/approved_drugs.sdf'

# Read the file into a supplier of `Mol` objects using the `Chem.SDMolSupplier()` function
# Each molecule in the SDF file is read as an RDKit `Mol` object
supplier = Chem.SDMolSupplier(approved_drugs_file)

molecules = []
fingerprints = []

# Iterate through `Mol` objects in `supplier`
for molecule in supplier:
    if molecule is None:  
        continue  # Skip invalid molecules
    molecules.append(molecule)  # Add valid molecule to the list
    fp = AllChem.GetMorganFingerprint(molecule, radius=3)  # Generate a Morgan fingerprint with a radius of 3 as an integer vector
    fingerprints.append(fp)  # Add fingerprint to the list

# Print the number of molecules and fingerprints generated
print("Number of molecules:", len(molecules))
print("Number of fingerprints:", len(fingerprints))

# Create a `MaxMinPicker` object to perform diversity picking
picker = MaxMinPicker()

RDLogger.logger().setLevel(RDLogger.WARNING)  # Re-enable RDKit warnings

#### STEP 2: Defining a distance function

To perform diversity picking, we need to calculate the "distance" between molecules. The distance function is derived from a similarity metric, such as the Dice coefficient:

$$Distance = 1 - Similarity$$

The cell below contains code that defines this function.

In [ ]:
def distance_ij(i, j, fps=fingerprints):
    """
    Calculate the distance between two fingerprints at indexes `i` and `j`.
    
    Parameters:
        i, j (int): Indexes of the fingerprints to compare.
        fps (list): List of RDKit fingerprint objects.
    
    Returns:
        float: Distance between the fingerprints.
    """
    similarity = DiceSimilarity(fps[i], fps[j])  # Compute Dice similarity
    return 1 - similarity  # Convert similarity to distance

# No output is expected from the execution of this cell

Let’s calculate the distance between two molecules at specified indexes and visualise them.

In [ ]:
# Select two molecule indexes to compare
i = 22
j = 111

# Calculate the distance between their fingerprints
dist = distance_ij(i, j, fps=fingerprints)
print(f'Distance between molecules {i} and {j}: {dist}')

# Retrieve the corresponding molecules
molecule_i = molecules[i]
molecule_j = molecules[j]

# Generate 2D coordinates for better visualisation
AllChem.Compute2DCoords(molecule_i)
AllChem.Compute2DCoords(molecule_j)

# Draw the molecules side by side for comparison
img = Draw.MolsToGridImage([molecule_i, molecule_j], 
                           legends=[f"Molecule {i}", f"Molecule {j}"])
img

#### STEP 4: Selecting diverse molecules with the `MaxMinPicker` object

Now that we have our fingerprints and distance function defined, we can use the `MaxMinPicker` object to select a subset of diverse molecules from the dataset.

In [ ]:
# Get the total number of fingerprints
n_fps = len(fingerprints)  

# Use the `MaxMinPicker` object to select 9 diverse molecule indexes
# The `seed` argument ensures the selection process is reproducible
pick_indexes = picker.LazyPick(distFunc=distance_ij,
                               poolSize=n_fps, 
                               pickSize=9, 
                               seed=920)

# Retrieve the molecules corresponding to the selected indexes
picked_molecules = []
for index in pick_indexes:
    picked_molecule = molecules[index]
    AllChem.Compute2DCoords(picked_molecule)  # Generate 2D coordinates for better visualisation
    picked_molecules.append(picked_molecule)

# Display the selected diverse molecules
img = Draw.MolsToGridImage(picked_molecules, legends=[f"Pick number {i+1}" for i in range(len(picked_molecules))])
img

##### Exercise 3: Exploring the effects of the `seed` and `distFunc` arguments

Complete the following tasks:

1. Experiment with the distance function. Modify the `distFunc` used in the `LazyPick` process, e.g. switch from Dice similarity to Tanimoto similarity. Do the picks differ based on the new distance function?
2. Change the seed by modifying the `seed` value in the `LazyPick` function. Observe how changing the seed affects the selected molecules.
3. Check how diverse the picked molecules are by calculating the average pairwise similarity between them. Use your distance function or RDKit’s built-in similarity functions to compute the similarities.

<details>
    <summary>
        Example solution
    </summary>

```python
def calculate_average_similarity(fps, similarity_func=DiceSimilarity):
    """
    Calculates the average pairwise similarity for a set of fingerprints 
    without using itertools.
    
    Parameters:
        fps (list): A list of RDKit fingerprint objects.
        similarity_func (function): Similarity function to compute pairwise similarity (default: DiceSimilarity).
    
    Returns:
        float: Average pairwise similarity.
    """
    total_similarity = 0.0
    num_pairs = 0
    for i in range(len(fps)):  # Loop through all pairs of fingerprints
        for j in range(i + 1, len(fps)):  # Ensure each pair is only compared once
            total_similarity += similarity_func(fps[i], fps[j])
            num_pairs += 1
    return total_similarity / num_pairs if num_pairs > 0 else 0.0  # Calculate and return the average similarity

picked_fps = [fingerprints[index] for index in pick_indexes]  # Extract fingerprints for the picked molecules

avg_similarity = calculate_average_similarity(picked_fps)  # Call the function

print("Average pairwise similarity:", avg_similarity)
```
</details>

In [ ]:
def distance_ij(i, j, fps=fingerprints):
    # Add code to define a new distance function

pick_indexes = picker.LazyPick(distFunc=distance_ij,
                               poolSize=n_fps,
                               pickSize=9,
                               seed=920)  # Change the `seed` value

picked_molecules = []
for index in pick_indexes:
    picked_molecule = molecules[index]
    AllChem.Compute2DCoords(picked_molecule)
    picked_molecules.append(picked_molecule)

img = Draw.MolsToGridImage(picked_molecules, 
                           legends=[f"Pick number {i+1}" for i in range(len(picked_molecules))])
img

# Add your solution code for task 3 here

----

## Discussion

That concludes our introduction to molecular similarity and wraps up the RDKit tutorial content. Feel free to add more code cells and experiment with the concepts learned.

With a solid understanding of molecular similarity, you are well-equipped to explore more advanced cheminformatics workflows.

You can click [here](#Contents) to return to the beginning and revisit any sections as needed.